# PySpark中的聚合函数

---

[好用的pyspark文档：sparkbyexamples](https://sparkbyexamples.com/)

1. 数据分析中常用`groupBy`分组函数，一般在分组函数分组后进行聚合统计。
1. PySpark 中的聚合函数可解决90%以上的数据分组统计问题。

In [1]:
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql import functions as F
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

## 创建数据

In [108]:
df_pd = pd.DataFrame({
    'id': ['001', '001', '001', '002', '002', '002'],
    'name': ['Bob', 'Bob', 'Bob', 'Tom', 'Tom', 'Tom'],
    'state': [1, 2, 2, 1, 2, 3],
    'date': ['20210101', '20210101', '20210201', '20210201', '20210101', '20210101']
})

In [109]:
df = spark.createDataFrame(df_pd)
df.show()

+---+----+-----+--------+
| id|name|state|    date|
+---+----+-----+--------+
|001| Bob|    1|20210101|
|001| Bob|    2|20210101|
|001| Bob|    2|20210201|
|002| Tom|    1|20210201|
|002| Tom|    2|20210101|
|002| Tom|    3|20210101|
+---+----+-----+--------+



## 统计每日不同状态比率

### 最简单实现方法

1. `F.countDistinct` 统计 `distinct` 值。
1. `F.count` 统计计数
1. `.alias` 取别名
1. `.toPandas()`转换为Pandas

In [10]:
df.groupBy('id','date').agg(
    (F.countDistinct('state') / F.count('state')).alias('distinct_rate')
).show()

+---+--------+------------------+
| id|    date|     distinct_rate|
+---+--------+------------------+
|002|20210101|               1.0|
|001|20210101|0.6666666666666666|
+---+--------+------------------+



In [4]:
df.groupBy('id', 'date').agg(
    (F.countDistinct('state') / F.count('state')).alias('distinct_rate')
).toPandas()

,id,date,distinct_rate
0,002,20210101,1.000000
1,001,20210101,0.666667


### 延伸

1. `F.collect_list` 把组中全部数据聚合至 `list`
1. `F.collect_set` 把组中全部数据聚合至 `set`
1. `F.size` 统计可迭代对象（list、set）的长度

In [7]:
df.groupBy('id','date').agg(
    F.collect_list('state').alias('state_list'),   # 不去重，所有数据
    F.collect_set('state').alias('state_set'),     # 去重
    F.size(F.collect_list('state')).alias('list_size'), 
    F.size(F.collect_set('state')).alias('set_size'),  
    (F.size(F.collect_set('state'))/F.size(F.collect_list('state'))).alias('distinct_rate'),
).toPandas()

,id,date,state_list,state_set,list_size,set_size,distinct_rate
0,002,20210101,"[1, 2, 3]","[1, 2, 3]",3,3,1.000000
1,001,20210101,"[1, 2, 2]","[1, 2]",3,2,0.666667


## 基本聚合函数

- `F.max`: 最大
- `F.avg`: 平均
- `F.min`: 最小
- `F.sum`: 求和

In [17]:
df.groupBy('id', 'date').agg(
    F.max('state').alias('max_state'),
    F.avg('state').alias('avg_state'),  # 怎么约束小数点
    F.min('state').alias('min_state'),
    F.sum('state').alias('sum_state'),
).toPandas()

,id,date,max_state,avg_state,min_state,sum_state
0,002,20210101,3,2.000000,1,6
1,001,20210101,2,1.666667,1,5


#### 以下未写完的东西，自己去查一下，然后补在这个文档中。

In [ ]:
F.sumDistinct
F.mean

In [14]:
df.groupBy('id','date').agg(
    F.sumDistinct('state').alias('唯一的大坏蛋汇总'),
    F.mean('state').alias('大坏蛋平均'),
).toPandas()

,id,date,唯一的大坏蛋汇总,大坏蛋平均
0,002,20210101,6,2.000000
1,001,20210101,3,1.666667


### Avg和Mean的区别
Mean Alias Avg.

## 粗略估计相关

countApproxDistinct（rsd):该函数是用来计算去重后的值的大约个数。

在进行sql优化时经常要统计某一列不同的值的个数，进而计算该列的选择性。但是在表体量很大，并且列上又没有索引时，count(distinct column)经常要花费很长的时间。pyspark提供了一个近似去重的新函数approx_count_distinct。这个函数的返回值是不准确的，不统计null值，但是胜在速度快。

 `rsd`： 该参数是用来规定值之间的相似度（精度值），值与值之间的相似度（精度）达到 rsd, 则将其看作是一样的值。**rsd越小说明值与值之间越相似**。控制在 0到0.4（不包括）之间。

demo1:
```python

           val rdd1 = sc.parallelize(List("a","b","c","d","e","a"),2)

           rdd1.countApproxDistinct(rsd=0.1) 输出结果为：Long = 5。

          

demo2：

                 val a = sc.parallelize(0 to 10000,20)

                 a.countApproxDistinct(rsd=0.1);  输出结果为：Long = 8224

                 a.countApproxDistinct(rsd=0.2);  输出结果为：Long = 10249

                 a.countApproxDistinct(rsd=0.3);  输出结果为： Long = 9618

                 a.countApproxDistinct(rsd=0.01);  输出结果为： Long = 9947

                 a.countApproxDistinct(rsd=0.001);  输出结果为： Long = 10001
```

In [23]:
df.groupBy('id','date').agg(
    F.approxCountDistinct('state',rsd=0.1).alias('粗略统计唯一的大坏蛋个数'),
).toPandas()

,id,date,粗略统计唯一的大坏蛋个数
0,002,20210101,3
1,001,20210101,2


In [25]:
df.groupBy('id','date').agg(
    F.countDistinct('state').alias('精确统计唯一的大坏蛋个数')
).toPandas()

,id,date,精确统计唯一的大坏蛋个数
0,002,20210101,3
1,001,20210101,2


## 正则匹配相关

F.regexp_extract

从指定的字符串列中提取与 Java 正则表达式匹配的特定组。如果正则表达式不匹配，或指定的组不匹配，则返回空字符串。

In [28]:
df = spark.createDataFrame([('100-200',)], ['str'])
df.show()
df.select(F.regexp_extract('str', r'(\d+)-(\d+)', 1).alias('d')).collect()

+-------+
|    str|
+-------+
|100-200|
+-------+



[Row(d='100')]

In [35]:
spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()
address = [(1,"14851 Jeffrey Rd","DE"),
    (2,"43421 Margarita St","NY"),
    (3,"13111 Siemon Ave","CA")]
dfreg =spark.createDataFrame(address,["id","address","state"])
dfreg.show()

+---+------------------+-----+
| id|           address|state|
+---+------------------+-----+
|  1|  14851 Jeffrey Rd|   DE|
|  2|43421 Margarita St|   NY|
|  3|  13111 Siemon Ave|   CA|
+---+------------------+-----+



您可以将列值替换为另一个字符串/子字符串的字符串。`regexp_replace()`使用Java 正则表达式进行匹配，如果正则表达式不匹配则返回空字符串.          
下面的示例将街道名称Rd值替换为列上的Road字符串address。

In [36]:
dfreg.withColumn('address', F.regexp_replace('address', 'Rd', 'Road')).show(truncate=False)

+---+------------------+-----+
|id |address           |state|
+---+------------------+-----+
|1  |14851 Jeffrey Road|DE   |
|2  |43421 Margarita St|NY   |
|3  |13111 Siemon Ave  |CA   |
+---+------------------+-----+



In [46]:
dfreg.filter(dfreg.address.rlike('14$')).collect()

[]

In [ ]:
F.col('').rlike ##不会
F.regexp_extract
F.regexp_replace

## 时间相关

```python
df.withColumn('test', F.date_format(col('Last_Update'),"yyyy/MM/dd")).show()

df.withColumn('date', F.date_format(col('Last_Update'),"yyyy-MM-dd").alias('ts').cast("date"))

# 把 timestamp 秒数（从1970年开始）转成日期格式 string
F.from_unixtime()

# unix_timestamp 把 日期 String 转换成 timestamp 秒数，是上面操作的反操作
F.unix_timestamp()
df1 = df.withColumn("unix_timestamp",F.unix_timestamp(df.TIME,'dd-MMM-yyyy HH:mm:ss.SSS z') + F.substring(df.TIME,-7,3).cast('float')/1000)

# timestamp 秒数转换成 timestamp type, 可以用 F.to_timestamp
```

In [ ]:
F.date_add| F.date_sub
F.add_months
F.date_format
F.datediff
F.dayofmonth  # 取月份中的一天
F.dayofweek   # 取一周中的一天
F.dayofyear   # 取一年中的一天

In [138]:
df2 = df.withColumn("Date", F.when(df.date == '20210101','2020-01-01')
                              .when(df.date == '20210201','2020-06-01')
                                 .otherwise(df.date))

In [139]:
df2.show()

+---+----+-----+----------+
| id|name|state|      Date|
+---+----+-----+----------+
|001| Bob|    1|2020-01-01|
|001| Bob|    2|2020-01-01|
|001| Bob|    2|2020-06-01|
|002| Tom|    1|2020-06-01|
|002| Tom|    2|2020-01-01|
|002| Tom|    3|2020-01-01|
+---+----+-----+----------+



In [176]:
# 加天数
df3 = df2.withColumn('Date',F.date_add(df2.Date, 7))
df3.show()

# 转换格式
df2.select(F.date_format('date','yyyy MM dd').alias('dateformat')).show()

# 加月份(统一加到20200608)
anniversary = df3.select(F.when(df3.Date == '2020-01-08', F.add_months(df3.Date,5))
                         .otherwise(df3.Date)
                         .alias('纪念日'))
anniversary.show()


# 计算相隔天数
anniversary.select("纪念日",
                   F.current_date().alias('当前时间'),
                   F.datediff(F.current_date(),anniversary["纪念日"])
                   .alias('在一起的时间')).show()




+---+----+-----+----------+
| id|name|state|      Date|
+---+----+-----+----------+
|001| Bob|    1|2020-01-08|
|001| Bob|    2|2020-01-08|
|001| Bob|    2|2020-06-08|
|002| Tom|    1|2020-06-08|
|002| Tom|    2|2020-01-08|
|002| Tom|    3|2020-01-08|
+---+----+-----+----------+

+----------+
|dateformat|
+----------+
|2020 01 01|
|2020 01 01|
|2020 06 01|
|2020 06 01|
|2020 01 01|
|2020 01 01|
+----------+

+----------+
|    纪念日|
+----------+
|2020-06-08|
|2020-06-08|
|2020-06-08|
|2020-06-08|
|2020-06-08|
|2020-06-08|
+----------+



,纪念日,当前时间,在一起的时间
0,2020-06-08,2021-06-30,387
1,2020-06-08,2021-06-30,387
2,2020-06-08,2021-06-30,387
3,2020-06-08,2021-06-30,387
4,2020-06-08,2021-06-30,387
5,2020-06-08,2021-06-30,387


## 选取位置

In [ ]:
F.first
F.last

In [75]:
df.groupBy('id','date').agg(
    F.first('state').alias('第一个大坏蛋'),
    F.last('state').alias('最后一个大坏蛋')
).toPandas()

,id,date,第一个大坏蛋,最后一个大坏蛋
0,002,20210101,1,3
1,001,20210101,1,2


## 统计量

In [ ]:
F.kurtosis
F.skewness
F.variance
F.var_pop
F.var_samp
F.stddev
F.stddev_pop
F.stddev_samp

In [77]:
df.groupBy('id','date').agg(
    F.kurtosis('state').alias('峰度'),
    F.skewness('state').alias('偏度'),
    F.variance('state').alias('方差'),
    F.var_pop('state').alias('总体方差'),
    F.var_samp('state').alias('样本方差'),
    F.stddev('state').alias('方差标准差'),
    F.stddev_pop('state').alias('总体标准差'),
    F.stddev_samp('state').alias('样本标准差')
).toPandas()

,id,date,峰度,偏度,方差,总体方差,样本方差,方差标准差,总体标准差,样本标准差
0,002,20210101,-1.5,0.000000,1.000000,0.666667,1.000000,1.00000,0.816497,1.00000
1,001,20210101,-1.5,-0.707107,0.333333,0.222222,0.333333,0.57735,0.471405,0.57735


# 进阶

## When

PySpark 支持按顺序检查多个条件并在第一个条件满足时返回值的方法，例如使用SQL 的 `case when`和`when().otherwise()`表达式，这些工作类似于`Switch`和`if then else`语句。

In [ ]:
F.when().when().otherwise()

In [96]:
df2 = df.withColumn("Date", F.when(df.date == '20210101','2020-01-01')
                                 .otherwise(df.date))

In [99]:
df2.show()

+---+----+-----+----------+
| id|name|state|      Date|
+---+----+-----+----------+
|001| Bob|    1|2020-01-01|
|001| Bob|    2|2020-01-01|
|001| Bob|    2|2020-01-01|
|002| Tom|    1|2020-01-01|
|002| Tom|    2|2020-01-01|
|002| Tom|    3|2020-01-01|
+---+----+-----+----------+



In [100]:
df2.select(F.date_add(df2.Date, 1).alias('next_date')).collect()

[Row(next_date=datetime.date(2020, 1, 2)),
 Row(next_date=datetime.date(2020, 1, 2)),
 Row(next_date=datetime.date(2020, 1, 2)),
 Row(next_date=datetime.date(2020, 1, 2)),
 Row(next_date=datetime.date(2020, 1, 2)),
 Row(next_date=datetime.date(2020, 1, 2))]

---